In [1]:
from duckberg import DuckBerg

## Init catalog config and connection properties

In [2]:
MINIO_URI = "http://minio:9000/"
MINIO_USER = "admin"
MINIO_PASSWORD = "password"

catalog_config: dict[str, str] = {
  "type": "rest",
  "uri": "http://iceberg-rest:8181/",
  "credentials": "admin:password",
  "s3.endpoint": MINIO_URI,
  "s3.access-key-id": MINIO_USER,
  "s3.secret-access-key": MINIO_PASSWORD
}

## Duckberg initialisation

In [3]:
db = DuckBerg(
     catalog_name="warehouse",
     catalog_config=catalog_config)

## List tables

In [4]:
tables = db.list_tables()
tables

['nyc.taxis']

## List partitions
To see what partitions you can use afterward in the partitions filters

In [5]:
partitions = db.list_partitions(table="nyc.taxis")
partitions

['payment_type']

## Query data - filter by partition
With usage of partition filter to read data just from files we need.

In [7]:
query = "SELECT * FROM 'nyc.taxis' WHERE payment_type = 1 AND trip_distance > 40 ORDER BY tolls_amount DESC"
df = db.select(sql=query).read_pandas()
df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2022-03-02 07:18:59+00:00,2022-03-02 09:21:20+00:00,1.0,80.17,5.0,N,132,265,1,250.0,0.00,0.0,0.10,54.35,0.3,306.00,0.0,1.25
1,2,2022-03-28 17:56:09+00:00,2022-03-28 20:26:28+00:00,1.0,151.16,4.0,N,132,265,1,540.5,1.00,0.5,0.00,54.30,0.3,597.85,0.0,1.25
2,2,2022-01-05 00:40:02+00:00,2022-01-05 02:28:48+00:00,4.0,84.18,5.0,N,132,265,1,400.0,0.00,0.0,10.00,48.30,0.3,459.85,0.0,1.25
3,1,2022-03-15 01:37:35+00:00,2022-03-15 03:37:34+00:00,1.0,78.20,5.0,N,132,265,1,200.0,1.25,0.0,50.00,48.20,0.3,299.75,0.0,1.25
4,2,2022-02-18 17:30:59+00:00,2022-02-18 20:27:41+00:00,1.0,61.20,3.0,N,138,138,1,203.0,1.00,0.0,63.18,47.15,0.3,315.88,0.0,1.25
5,2,2022-02-27 16:38:02+00:00,2022-02-27 17:47:05+00:00,1.0,67.88,4.0,N,132,265,1,128.0,0.00,0.5,20.00,47.15,0.3,197.20,0.0,1.25
6,1,2022-01-06 18:33:09+00:00,2022-01-06 20:51:46+00:00,1.0,139.80,5.0,N,132,265,1,375.0,0.00,0.0,0.10,45.30,0.3,420.70,0.0,0.00
7,2,2022-02-06 16:36:38+00:00,2022-02-06 18:30:41+00:00,1.0,86.50,5.0,N,132,265,1,425.0,0.00,0.0,0.00,44.85,0.3,470.15,0.0,0.00
8,2,2022-03-11 14:34:16+00:00,2022-03-11 16:20:04+00:00,3.0,84.33,5.0,N,90,265,1,332.5,0.00,0.0,76.01,44.75,0.3,456.06,2.5,0.00
9,2,2022-03-31 06:56:08+00:00,2022-03-31 08:34:13+00:00,1.0,62.26,5.0,N,132,265,1,310.0,0.00,0.0,40.00,44.30,0.3,394.60,0.0,0.00


### Query data - Old way

In [9]:
query = "SELECT * FROM 'nyc.taxis' WHERE trip_distance > 40 ORDER BY tolls_amount DESC"
df = db.select(sql=query, table="nyc.taxis", partition_filter="payment_type = 1").read_pandas()
df.head(10)

/opt/conda/lib/python3.11/site-packages/pyiceberg/avro/decoder.py:184: UserWarning: Falling back to pure Python Avro decoder, missing Cython implementation
  warnings.warn("Falling back to pure Python Avro decoder, missing Cython implementation")


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2022-03-02 07:18:59+00:00,2022-03-02 09:21:20+00:00,1.0,80.17,5.0,N,132,265,1,250.0,0.00,0.0,0.10,54.35,0.3,306.00,0.0,1.25
1,2,2022-03-28 17:56:09+00:00,2022-03-28 20:26:28+00:00,1.0,151.16,4.0,N,132,265,1,540.5,1.00,0.5,0.00,54.30,0.3,597.85,0.0,1.25
2,2,2022-01-05 00:40:02+00:00,2022-01-05 02:28:48+00:00,4.0,84.18,5.0,N,132,265,1,400.0,0.00,0.0,10.00,48.30,0.3,459.85,0.0,1.25
3,1,2022-03-15 01:37:35+00:00,2022-03-15 03:37:34+00:00,1.0,78.20,5.0,N,132,265,1,200.0,1.25,0.0,50.00,48.20,0.3,299.75,0.0,1.25
4,2,2022-02-18 17:30:59+00:00,2022-02-18 20:27:41+00:00,1.0,61.20,3.0,N,138,138,1,203.0,1.00,0.0,63.18,47.15,0.3,315.88,0.0,1.25
5,2,2022-02-27 16:38:02+00:00,2022-02-27 17:47:05+00:00,1.0,67.88,4.0,N,132,265,1,128.0,0.00,0.5,20.00,47.15,0.3,197.20,0.0,1.25
6,1,2022-01-06 18:33:09+00:00,2022-01-06 20:51:46+00:00,1.0,139.80,5.0,N,132,265,1,375.0,0.00,0.0,0.10,45.30,0.3,420.70,0.0,0.00
7,2,2022-02-06 16:36:38+00:00,2022-02-06 18:30:41+00:00,1.0,86.50,5.0,N,132,265,1,425.0,0.00,0.0,0.00,44.85,0.3,470.15,0.0,0.00
8,2,2022-03-11 14:34:16+00:00,2022-03-11 16:20:04+00:00,3.0,84.33,5.0,N,90,265,1,332.5,0.00,0.0,76.01,44.75,0.3,456.06,2.5,0.00
9,2,2022-03-31 06:56:08+00:00,2022-03-31 08:34:13+00:00,1.0,62.26,5.0,N,132,265,1,310.0,0.00,0.0,40.00,44.30,0.3,394.60,0.0,0.00


## Query data - group by

In [11]:
query = "SELECT passenger_count, AVG(tip_amount) as tip_amount_sum FROM 'nyc.taxis' WHERE payment_type = 1 GROUP BY passenger_count"
df = db.select(sql=query).read_pandas()
df.head(10)

/opt/conda/lib/python3.11/site-packages/pyiceberg/avro/decoder.py:184: UserWarning: Falling back to pure Python Avro decoder, missing Cython implementation
  warnings.warn("Falling back to pure Python Avro decoder, missing Cython implementation")


,passenger_count,tip_amount_sum
0,1.0,3.107739
1,2.0,3.412747
2,3.0,3.298482
3,0.0,3.017754
4,5.0,3.157198
5,6.0,3.220024
6,4.0,3.456813
7,7.0,9.867857
8,8.0,8.916667
9,9.0,10.012000


## Query data - group by - Old way

In [10]:
query = "SELECT passenger_count, AVG(tip_amount) as tip_amount_sum FROM 'nyc.taxis' GROUP BY passenger_count"
df = db.select(table="nyc.taxis", partition_filter="payment_type = 1", sql=query).read_pandas()
df.head(10)

/opt/conda/lib/python3.11/site-packages/pyiceberg/avro/decoder.py:184: UserWarning: Falling back to pure Python Avro decoder, missing Cython implementation
  warnings.warn("Falling back to pure Python Avro decoder, missing Cython implementation")


,passenger_count,tip_amount_sum
0,1.0,3.107739
1,2.0,3.412747
2,3.0,3.298482
3,0.0,3.017754
4,5.0,3.157198
5,6.0,3.220024
6,4.0,3.456813
7,7.0,9.867857
8,8.0,8.916667
9,9.0,10.012000
